# 1) + 2)

In [30]:
import numpy as np

# shamelessly stolen from stack overflow
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

Phi = np.matrix([
  [1, 0.8, 0.64, 0.512],
  [1, 1, 1, 1],
  [1, 1.2, 1.44, 1.728],
  [1, 1.4, 1.96, 2.744],
  [1, 1.6, 2.56, 4.096],
])

Phi_T = Phi.transpose()
I = np.identity(4) * 2
z = np.matrix([24, 20, 10, 13, 12]).transpose()

left = np.linalg.inv(Phi_T * Phi + I)
right = Phi_T * z
w = left * right

def print_first_question_matrices():
  for matrix in [Phi, Phi_T, I, z, left, right, w]:
    print(bmatrix(matrix))
    print("---")

def print_second_question_results():
  # compute the training RMSE for the learnt regression model
  forecasts = Phi * w
  print("z hat:")
  print(bmatrix(forecasts))
  n = len(forecasts)
  rmse = np.sqrt(np.sum(np.square(forecasts - z)) / n)
  print("Training RMSE: {}".format(rmse))

In [ ]:
print_first_question_matrices()

In [31]:
print_second_question_results()

z hat:
\begin{bmatrix}
  11.35086463\\
  12.35246259\\
  13.19923625\\
  13.8287433\\
  14.17854143\\
\end{bmatrix}
Training RMSE: 6.843294892094978


# 3)

In [36]:
# activation function f is e^0.1x

def activation(x):
  return np.exp(0.1 * x)

x_0 = np.matrix([0.8, 1, 1.2]).transpose()
w_1 = np.matrix([[1, 1, 1], [1, 1, 1], [1, 1, 1]]).transpose()
b_1 = np.matrix([[1, 1, 1]]).transpose()

z_1 = w_1 * x_0 + b_1
print(bmatrix(z_1))
x_1 = activation(z_1)
print(bmatrix(x_1))

w_2 = np.matrix([[1, 1], [1, 1], [1, 1]]).transpose()
b_2 = np.matrix([[1, 1]]).transpose()

z_2 = w_2 * x_1 + b_2
print(bmatrix(z_2))
x_2 = activation(z_2)
print(bmatrix(x_2))

\begin{bmatrix}
  4.\\
  4.\\
  4.\\
\end{bmatrix}
\begin{bmatrix}
  1.4918247\\
  1.4918247\\
  1.4918247\\
\end{bmatrix}
\begin{bmatrix}
  5.47547409\\
  5.47547409\\
\end{bmatrix}
\begin{bmatrix}
  1.72900727\\
  1.72900727\\
\end{bmatrix}
